In [1]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm
import json
import re
import pandas as pd

In [2]:
# importing the module
import json
  
# Opening JSON file
f_train = open('json_datasets/train.json', 'r')
raw_train = json.load(f_train)
f_train.close()

f_test = open('json_datasets/test.json', 'r')
raw_test = json.load(f_test)
f_test.close()

In [3]:
def removeQuotes(lst):
    ret = []
    for s in lst:
        if s != '``' and s != "''":
            ret.append(s)
    return ret

def camelCaseSplit(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    d = [m.group(0) for m in matches]
    new_d = []
    for token in d:
        token = token.replace('(', '')
        token = token.replace(')', '')
        token_split = token.split('_')
        for t in token_split:
            #new_d.append(t.lower())
            new_d.append(t)
    return new_d

In [4]:
def g2tPreprocess(raw):
    df = []
    for item in raw:
        graph = 'g2t:'
        for relation in item['relations']:
            graph += ' <H> ' + ' '.join(removeQuotes(relation[0])) + ' <R> '
            graph += ' '.join(camelCaseSplit(relation[1])) + ' <T> '
            graph += ' '.join(removeQuotes(relation[2]))

        ents = [' '.join(removeQuotes(entity)) for entity in item['entities']]
        text = item['text']
        for i in range(len(ents)):
            text = text.replace('<ENT_'+str(i)+'>', ents[i])
        sample = [graph, text]
        df.append(sample)
    return pd.DataFrame(df, columns=['source_text', 'target_text'])

In [5]:
train_df = g2tPreprocess(raw_train)
test_df = g2tPreprocess(raw_test)
print(test_df)

                                            source_text  \
0     g2t: <H> Abilene Regional Airport <R> city Ser...   
1     g2t: <H> Abilene Regional Airport <R> city Ser...   
2     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
3     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
4     g2t: <H> Adolfo Suárez Madrid–Barajas Airport ...   
...                                                 ...   
4923  g2t: <H> Twilight ( band ) <R> genre <T> Black...   
4924  g2t: <H> Twilight ( band ) <R> genre <T> Black...   
4925  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   
4926  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   
4927  g2t: <H> Uruguay <R> leader Name <T> Raúl Fern...   

                                            target_text  
0     Abilene , Texas is served by the Abilene Regio...  
1     Abilene Regional Airport serves the city of Ab...  
2     Adolfo Suárez Madrid–Barajas Airport can be fo...  
3     Adolfo Suárez Madrid–Barajas Airport is locate...  
4

In [6]:
pip install --upgrade simplet5

Note: you may need to restart the kernel to use updated packages.


In [7]:
# import
from simplet5 import SimpleT5

# instantiate
model = SimpleT5()

# load (supports t5, mt5, byT5 models)
model.from_pretrained("t5","t5-base")

# train
# model.train(train_df=train_df, # pandas dataframe with 2 columns: source_text & target_text
#             eval_df=test_df, # pandas dataframe with 2 columns: source_text & target_text
#             source_max_token_len = 512, 
#             target_max_token_len = 128,
#             batch_size = 8,
#             max_epochs = 5,
#             use_gpu = True,
#             outputdir = "outputs",
#             early_stopping_patience_epochs = 0,
#             precision = 32
#             )



Global seed set to 42


In [17]:

# load trained T5 model
model.load_model("t5","./local/simplet5", use_gpu=False)

In [8]:
f_dev = open('json_datasets/dev.json', 'r')
raw_dev = json.load(f_dev)
f_dev.close()

In [10]:
pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-zds5y0el
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-zds5y0el
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312215 sha256=4cc8ec30fb183efcc04aee46289bf02d3106bb29ad341f27150bfc40430c5294
  Stored in directory: /tmp/pip-ephem-wheel-cache-ljwg8jbe/wheels/6f/c9/51/e266f0496048c16686e133d8e33644d692931a356bfb372aae
Successfully built pycocoevalcap


In [15]:
dev_df['source_text'][0]

'g2t: <H> Aarhus <R> leader Name <T> Jacob Bundsgaard'

In [22]:
# from pycocoevalcap.bleu.bleu import Bleu
#bleu = Bleu(4)

dev_df = g2tPreprocess(raw_dev)
# print(dev_df)

#hyp = [model.predict(dev_df['source_text'][i])[0] for i in range(len(dev_df))]

hyp = model.predict(dev_df['source_text'][0])
# ret = bleu.compute_score(dev_df['target_text'], hyp)
print(dev_df['source_text'][0])
print(hyp[0])
# print(dev_df['source_text'][50])
# print(model.predict(dev_df['source_text'][50]))

g2t: <H> Aarhus <R> leader Name <T> Jacob Bundsgaard
Jacob Bundsgaard is the leader of Aarhus.


In [34]:
from nltk.translate.bleu_score import sentence_bleu

def g2t_metrics(predict,truth):
    BLEU_1 = 0
    BLEU_2 = 0
    BLEU_3 = 0
    BLEU_4 = 0

    for pred, tth in zip(predict, truth):
        b1_1, b2_1, b3_1, b4_1 = BLEU_score(tth, pred)
        BLEU_1 += b1_1
        BLEU_2 += b2_1
        BLEU_3 += b3_1
        BLEU_4 += b4_1

    BLEU_1 = BLEU_1 / len(truth)
    BLEU_2 = BLEU_2 / len(truth)
    BLEU_3 = BLEU_3 / len(truth)
    BLEU_4 = BLEU_4 / len(truth)
    
    print('BLEU_1: %s'%(BLEU_1))
    print('BLEU_2: %s'%(BLEU_2))
    print('BLEU_3: %s'%(BLEU_3))
    print('BLEU_4: %s'%(BLEU_4))
    
    return BLEU_1, BLEU_2, BLEU_3, BLEU_4
    
def BLEU_score(truth, predict):
    truth = truth.split(' ')
    truth = [truth]
    predict = predict.split(' ')
    BLEU_1 = sentence_bleu(truth, predict, weights=(1, 0, 0, 0))
    BLEU_2 = sentence_bleu(truth, predict, weights=(0.5, 0.5, 0, 0))
    BLEU_3 = sentence_bleu(truth, predict, weights=(0.33, 0.33, 0.33, 0))
    BLEU_4 = sentence_bleu(truth, predict, weights=(0.25, 0.25, 0.25, 0.25))  
    return BLEU_1, BLEU_2, BLEU_3, BLEU_4

In [35]:
g2t_metrics(hyp, dev_df['target_text'])

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU_1: 0.5897240751827024
BLEU_2: 0.4933572212312753
BLEU_3: 0.4366977576493366
BLEU_4: 0.4055509401240295


(0.5897240751827024,
 0.4933572212312753,
 0.4366977576493366,
 0.4055509401240295)